In [2]:
%matplotlib inline
ds = clean_xlsx_dataset(os.path.join(SOURCE_PATH, 'datasets/MoWTe2/20170725_MoWTe2.xlsx'))
from arpes.config import CONFIG
CONFIG['WORKSPACE'] = 'MoWTe2'

In [3]:
delay_scans_for_map = {
    'f00073': ['f00074', 'f00075', 'f00076', 'f00077', 'f00078', 'f00079', 'f00080', 'f00081', 'f00082', 'f00083', 'f00084'],
#    'f00055': ['f00056', 'f00057', 'f00058'],
}
ds_map = ds.loc['f00073']

In [ ]:
cropped_map = load_dataset(ds_map.id).sel(polar=slice(20.5, None))
cropped_map.sel(eV=slice(-0.03, 0.03)).sum('eV').plot()

# Notes on symmetry

The symmetry point in this dataset seems to be at around polar=28. The gamma point is out of the edge of the scan, we can infer where it should be in phi by looking at an earlier map.

In [ ]:
reference_map = load_dataset(ds.loc['f00032', ('id',)])
reference_map.sel(eV=slice(-0.03, 0.03)).sum('eV').plot()

#save_dataset(reference_map, force=True)

# Reference map

From the reference map f00031 and f00032, the distance from teh nominal gamma point to the edge of the arc at the pocket looks to be about 24 degrees

We will nominally pin the value of the Gamma point in f00073 to be (phi, beta) = (-22, 27)

In [ ]:
cropped_map.attrs['symmetry_points'] = {
    'G': {'phi': -22, 'polar': 27},
}
from arpes.analysis import d2_along_axis
from arpes.pipeline import pipeline, compose
from arpes.pipelines import convert_scan_to_kspace_no_corr
from arpes.utilities import polar_offset, phi_offset

In [ ]:
#kmap = load_dataset(convert_scan_to_kspace_no_corr(cropped_map))
#arr = convert_scan_to_kspace_no_corr(cropped_map, force=True)
arr.sel(eV=0, method='nearest').plot()

In [ ]:
sarr = boxcar_filter_arr(arr, size={'eV': 0.1, 'kx': 0.02}, n=3, default_size=1)
carr = curvature(sarr, directions=['kx', 'eV'], alpha=10)
carr.values[np.isnan(carr.values)] = 0
carr.values[carr.values > 0] = 0
carr.values = -carr.values + 0.01
cutoff = np.percentile(carr.values, 97)
low_cutoff = np.percentile(carr.values, 10)
carr.values[carr.values > cutoff] = cutoff
carr.values[carr.values < low_cutoff] = 0
carr = carr.transpose('kx', 'ky', 'eV')

In [ ]:
make_bokeh_tool(carr.sel(eV=slice(-0.6, 0)))

In [ ]:
narr = arr.copy()
narr.values[narr.values != narr.values] = 0
narr = narr.transpose('ky', 'eV', 'kx')
#make_bokeh_tool(narr)

In [ ]:
kmap.sel(eV=-0.2, method='nearest').plot()

# Convert delay scans to k-space

In [18]:
# Go and attach symmetry points for all the delay scans from the parent map
for k, vs in delay_scans_for_map.items():
    ref_dataset = load_dataset(ds.loc[k, ('id',)])
    for v in vs:
        #delay_scan = load_dataset(ds.loc[v, ('id',)])
        #delay_scan.attrs['symmetry_points'] = dict(ref_dataset.attrs['symmetry_points'])
        #save_dataset(delay_scan, force=True)
        simple_convert_tr(ds.loc[v, ('id',)])

/Users/chstansbury/PycharmProjects/python-arpes/arpes/io.py:145: UserWarning: Could not fetch cleave information.
  warnings.warn('Could not fetch cleave information.')


In [135]:
#f81 = load_dataset(simple_convert_tr(ds.loc['f00081', ('id',)]))
#f82 = load_dataset(simple_convert_tr(ds.loc['f00082', ('id',)]))
#f83 = load_dataset(simple_convert_tr(ds.loc['f00083', ('id',)]))
#f84 = load_dataset(simple_convert_tr(ds.loc['f00084', ('id',)]))
large_map_at_t0 = load_dataset(ds.loc['f00031', ('id',)])
large_map = load_dataset(ds.loc['f00032', ('id',)])
#f81_phi = load_dataset(ds.loc['f00081', ('id',)]).sum('cycle', keep_attrs=True)
#f82_phi = load_dataset(ds.loc['f00082', ('id',)]).sum('cycle', keep_attrs=True)
#f83_phi = load_dataset(ds.loc['f00083', ('id',)]).sum('cycle', keep_attrs=True)
#f84_phi = load_dataset(ds.loc['f00084', ('id',)]).sum('cycle', keep_attrs=True)
#f31_phi = load_dataset(ds.loc['f00031', ('id',)])
#f32_phi = load_dataset(ds.loc['f00032', ('id',)])


/Users/chstansbury/PycharmProjects/python-arpes/arpes/io.py:145: UserWarning: Could not fetch cleave information.
  warnings.warn('Could not fetch cleave information.')


In [1]:
from arpes.pipeline import compose, pipeline
from arpes.pipelines import tr_prep_scan_simple

simple_convert_tr = compose(tr_prep_scan_simple, convert_scan_to_kspace_no_corr)

In [127]:
def simple_curvature(arr):
    arr_s = boxcar_filter_arr(arr, size={'eV': 0.05, 'kp': 0.02}, n=4)
    arr_c = curvature(arr_s, directions=['eV', 'kp'], alpha=0.000000001) * 1000
    arr_c.values = -arr_c.values
    arr_c.values[arr_c.values <= 0] = 0
    return arr_c


def simple_curvature_phi(arr, size=None, alpha=0.000001):
    if size is None:
        size = {'eV': 0.05, 'phi': 2}
    arr_s = boxcar_filter_arr(arr, size=size, n=4)
    arr_c = curvature(arr_s, directions=['eV', 'phi'], alpha=alpha) * 1000
    arr_c.values = -arr_c.values
    arr_c.values[arr_c.values <= 0] = 0
    return arr_c



In [140]:
f83_phi_c = simple_curvature_phi(f83_phi)
# f81_phi alpha = 0.000001, size=None # defaults
# f82_phi alpha = 0.000001, size=None # defaults
# f83 defaults work reasonably too
make_bokeh_tool(f83_phi_c.transpose('phi', 'eV', 'delay'))

In [147]:
make_bokeh_tool(f83)

In [162]:
sscans = ds[ds['PrPo'] == 's']
sscans[sscans['type'] == 'map']
#ds[ds.type == 'map']

,time,type,path,scan,Theta,Beta,Phi,Delay (ps),x,y,...,Lens Mode,binning,Pass Energy,hv,W_func,frames,id,location,polar_offset,phi_offset
file,,,,,,,,,,,,,,,,,,,,,
f57748,10% W,map,20170724_MoW_10Te/fits/20170725_57748.fits,"Beta Scan to see Fermi surface, s pol",0.0,-10:0.5:20,0,0,1.501,34.658,...,6,2.0,10.0,5.93,5.18,100.0,6db8265c-b3c2-11e7-92ca-6c40088ef8e6,ALG-MC,27.0,-22.0
f00002,19% W,map,20170724_MoW_10Te/fits/20170725_00002.fits,"Beta Scan to see Fermi surface, on better spot...",-6.0,-18:1:10,0,Blocked,1.057,35.540,...,6,NaN,NaN,NaN,4.20,NaN,6d1a05dc-b3c2-11e7-90f3-6c40088ef8e6,ALG-MC,NaN,NaN
f00007,19% W,map,20170724_MoW_10Te/fits/20170725_00007.fits,"Beta Scan on new spot to see brillouin zone, s...",-2.0,0:0.5:32,15.246,Blocked,1.215,35.040,...,6,NaN,NaN,NaN,4.20,NaN,6d21592c-b3c2-11e7-9bc2-6c40088ef8e6,ALG-MC,NaN,NaN
f00008,19% W,map,20170724_MoW_10Te/fits/20170725_00008.fits,Beta Scan PE 5 on new domain,4.0,-15:0.5:20,105,Blocked,1.768,34.580,...,6,NaN,NaN,NaN,4.20,NaN,6d23c6be-b3c2-11e7-ac32-6c40088ef8e6,ALG-MC,NaN,NaN
f00009,19% W,map,20170724_MoW_10Te/fits/20170725_00009.fits,Beta Scan PE 20,4.0,-15:0.5:25,105,Blocked,1.768,34.580,...,6,NaN,NaN,NaN,4.20,NaN,6d258054-b3c2-11e7-be5c-6c40088ef8e6,ALG-MC,NaN,NaN
f00011,16:00:00,map,20170724_MoW_10Te/fits/20170727_00011.fits,redo above,4.0,-15:0.5:25,105,Blocked,1.768,34.580,...,6,NaN,NaN,NaN,4.20,NaN,6d281af8-b3c2-11e7-976f-6c40088ef8e6,ALG-MC,NaN,NaN
f00012,16:50:00,map,20170724_MoW_10Te/fits/20170727_00012.fits,"Search for gamma, s polarized",-3.0,-15:.05:25,125,blocked,1.088,34.478,...,6,NaN,NaN,NaN,4.20,NaN,6d299c5c-b3c2-11e7-a7e7-6c40088ef8e6,ALG-MC,NaN,NaN
f00014,18:15:00,map,20170724_MoW_10Te/fits/20170727_00014.fits,Same but with s,-4.0,-15:0.5:21,146,blocked,1.044,34.535,...,6,NaN,NaN,NaN,4.20,NaN,6d2cb510-b3c2-11e7-ba25-6c40088ef8e6,ALG-MC,NaN,NaN
f00015,18:15:00,map,20170724_MoW_10Te/fits/20170727_00015.fits,rotated back by 15 in phi,-3.0,-15:0.5:21,130,blocked,1.043,34.530,...,6,NaN,NaN,NaN,4.20,NaN,6d2ee27a-b3c2-11e7-b253-6c40088ef8e6,ALG-MC,0.0,4.0


In [165]:
make_bokeh_tool(load_dataset(ds.loc['f00013', ('id',)]))

/Users/chstansbury/PycharmProjects/python-arpes/arpes/io.py:145: UserWarning: Could not fetch cleave information.
  warnings.warn('Could not fetch cleave information.')


In [159]:
make_bokeh_tool(f41_phi)

/Users/chstansbury/PycharmProjects/python-arpes/arpes/io.py:145: UserWarning: Could not fetch cleave information.
  warnings.warn('Could not fetch cleave information.')


['BITPIX',
 'BRF',
 'DEVNM_0',
 'DR',
 'Delay_ps',
 'EN_0_0',
 'EN_0_1',
 'HOST',
 'HOST_IP',
 'LMOTOR0',
 'LMOTOR1',
 'LMOTOR2',
 'LMOTOR3',
 'LMOTOR4',
 'LMOTOR5',
 'LMOTOR6',
 'LOFFSET0',
 'LOFFSET1',
 'LOFFSET2',
 'LOFFSET3',
 'LOFFSET4',
 'LOFFSET5',
 'LOFFSET6',
 'LWLVLPN',
 'LWLVNM',
 'MMX0_0',
 'MMX0_2',
 'MMX1_1',
 'MMX2_0',
 'MMX2_2',
 'MMX3_3',
 'MMX4_4',
 'MMX5_5',
 'MMX6_6',
 'NAXIS',
 'NMSBDV0',
 'NM_0_0',
 'NM_0_1',
 'N_0_0',
 'N_0_1',
 'P',
 'PMOTOR0',
 'PMOTOR1',
 'PMOTOR2',
 'PMOTOR3',
 'PMOTOR4',
 'PMOTOR5',
 'PMOTOR6',
 'PO',
 'Phi',
 'PrPo',
 'Pr_FWHM_um',
 'PuPol',
 'Pu_FWHM_um',
 'Pu_Power_mW',
 'Pump_energy_uJcm2',
 'SCNTYP0',
 'SFBA_0',
 'SFBE0',
 'SFE_0',
 'SFFR_0',
 'SFIM0',
 'SFKE0_0',
 'SFLNM0',
 'SFPEV_0',
 'SFPE_0',
 'SFRGN0',
 'SFX0_0',
 'SFX1_0',
 'SFY0_0',
 'SFY1_0',
 'SF_HV',
 'SF_KE0',
 'SF_MODEL',
 'SF_NUM',
 'SF_PPEV',
 'SF_SER',
 'SMOTOR0',
 'SMOTOR1',
 'SMOTOR2',
 'SMOTOR3',
 'SMOTOR4',
 'SMOTOR5',
 'SOFFSET0',
 'SOFFSET1',
 'SOFFSET2',
 'SOFFSET